In [3]:
import json
from datetime import datetime
from dateutil import relativedelta

import pandas as pd
import sqlite3

# Pending Items

In [4]:
with sqlite3.connect("p-data/brem.db") as conn:
    df_pend = pd.read_sql_query("SELECT * from pending order by due_date", conn)
    df_acct = pd.read_sql_query("SELECT * from account order by [type], [name]", conn)

df_pend = df_pend.merge(df_acct[['id','name']], left_on='acct_to', right_on = 'id', how='left', suffixes=['','_y'])

df_pend['due'] = (pd.to_datetime(df_pend.due_date) - datetime.now()).dt.days

df_pend[['id','due', 'name','amount', 'info']]


,id,due,name,amount,info
0,74,-14,Essex Skating Club,95.00,Transfer to Ice House
1,7,2,JCP&L,24.18,
2,61,5,Costco Credit Card,1801.55,
3,83,11,NP School Lunch,0.00,
4,101,12,Association,471.00,Mail a check tot he new management
5,3,12,Chase Visa Xiao,1271.35,
6,29,12,Mortgage 66,3366.08,
7,80,12,Amazon Visa,331.02,
8,118,14,Pediatric Eye Physicians,0.00,
9,119,21,Skating Competition (not used),187.20,


# Make Payment

In [4]:
pending_id = 101
pending = df_pend.loc[df_pend.id == pending_id].iloc[0]
print(f"Last paid: {pending['name']}, {pending['amount']}, {pending['last_date']}")

next_due = datetime.fromisoformat(pending['due_date']) + relativedelta.relativedelta(months=1)
print(f"Next: {next_due}")
print("----------------------")

########################################
# Record
rec = {
    "date": f"{datetime.now()}",
    "acct_from" : pending['acct_from'],
    "acct_to" : pending['acct_to'],
    "amount": 471, #pending['amount'],
    "info": 'Mail a check tot he new management',#pending['info']
}
qry_rec = f"""insert into record 
        values(NULL,
        '{rec['date']}',
        {rec['acct_from']}, {rec['acct_to']},
        {rec['amount']},--amount
        '{rec['info']}');"""

# Remind
if next_due:
    remind = {
        "last_date": rec['date'],
        'due_date': f"{next_due}",
        "alert" : pending['alert'],
        "acct_from" : rec['acct_from'],
        "acct_to" : rec['acct_to'],
        "amount": rec['amount'],
        "info": rec['info']
    }
    qry_rem = f"""update pending set last_date ='{remind['last_date']}', due_date = '{remind['due_date']}',
                amount = {remind['amount']}, info ='{remind['info']}'
                where id = {pending_id};"""
else:
    qry_rem = f"""delete from pending where id = {pending_id}"""



print('Record: ', qry_rec)
print('Remind: ', qry_rem)


Last paid: Association, 471.0, 2022-05-05 11:05:52.392902
Next: 2022-07-01 00:00:00
----------------------
Record:  insert into record 
        values(NULL,
        '2022-05-06 08:37:15.141709',
        35, 37,
        471,--amount
        'Mail a check tot he new management');
Remind:  update pending set last_date ='2022-05-06 08:37:15.141709', due_date = '2022-07-01 00:00:00',
                amount = 471, info ='Mail a check tot he new management'
                where id = 101;


In [4]:
# Execute query

with sqlite3.connect("p-data/brem.db") as conn:
    conn.execute(qry_rec)
    conn.commit()
    
    conn.execute(qry_rem)
    conn.commit()
    

# All Account


In [18]:
with sqlite3.connect("p-data/brem.db") as conn:
    df_acct = pd.read_sql_query("SELECT * from account order by [type], [name]", conn)

df_acct

,id,name,type,Create,url,info
0,84,Apple,*Notes,2013-05-29 00:00:00.000,None,"My iPad:Chengyou66\n\nITune: gmail, X6-6cy. Sa..."
1,187,Asana,*Notes,2019-10-24 00:00:00.000,https://app.asana.com,Xcy@gc2019
2,89,Baidu,*Notes,2013-08-18 00:00:00.000,http://wenku.baidu.com,"gmail, 6-6"
3,83,BlackBerry,*Notes,2013-05-29 00:00:00.000,None,"id: gc email, fw0rsf\nx-cy"
4,88,Code Encript,*Notes,2013-08-01 00:00:00.000,None,zip file pwd: Guy Carpenter
...,...,...,...,...,...,...
192,156,NY online,Tax,2017-04-11 00:00:00.000,https://my.ny.v/sreg/Login?APP=nyappdtfotc&TYP...,gmail\nchengyou3\nx6-6cy\n\nteacher-zhou\nempl...
193,19,Tax retur,Tax,2003-01-01 03:00:00.000,None,None
194,133,TurboTax,Tax,2016-02-08 00:00:00.000,https://turbotax.intuit.com,gmail\nXcy@2019\nMingyao 1098: https://student...
195,168,TurboTax Mingyao,Tax,2018-04-11 00:00:00.000,https://turbotax.intuit.com,Did't use for 2017. used 1040.com free file\n\...


In [17]:
# df_acct.type.drop_duplicates()
# df_acct[df_acct.type == 'Apartment']
print(df_acct[df_acct.id == 37].iloc[0])

id                                                       37
name                                            Association
type                                              Apartment
Create                              2004-06-29 03:00:00.000
url       https://miamimanagement.association-account.co...
info      $281/mo\n\nstart since change new management\n...
Name: 23, dtype: object


In [13]:
with sqlite3.connect("p-data/brem.db") as conn:
    conn.execute("""update account set name = 'Association'
        where id = 37""")
    conn.commit()

# Account History

In [5]:
acct_id = 37
with sqlite3.connect("p-data/brem.db") as conn:
    acct_hist = pd.read_sql_query(f"""SELECT * from record where acct_to = 37 order by date desc""", conn)

acct_hist    

,id,date,acct_from,acct_to,amount,info
0,3060,2022-05-05 11:05:52.392902,35,37,471.0,Mail a check tot he new management
1,3052,2022-04-07 00:00:00.000,35,37,474.0,new website. $471 +3
2,3045,2022-03-04 00:00:00.000,35,37,474.0,S086C142
3,3038,2022-02-04 00:00:00.000,35,37,462.0,S037DF3D
4,3025,2022-01-01 00:00:00.000,35,37,462.0,S03FE625
...,...,...,...,...,...,...
216,373,2004-10-26 03:00:00.000,1,37,281.0,
217,352,2004-09-24 03:00:00.000,1,37,281.0,
218,334,2004-08-26 03:00:00.000,1,37,281.0,
219,310,2004-07-26 03:00:00.000,35,37,281.0,


In [12]:
from brem import open_google

open_google()